Using DOcplex module to generate Hamiltonian, the model was not changed.

In [1]:
from qiskit import Aer
from qiskit.aqua.algorithms import VQE, ExactEigensolver, QAOA
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua import QuantumInstance
from qiskit.aqua import aqua_globals
from qiskit.circuit.library import RealAmplitudes
from qiskit.visualization import circuit_visualization
from docplex.mp.model import Model
from qiskit.optimization.applications.ising import docplex
# from qiskit import IBMQ
# IBMQ.load_account()
import numpy as np
import csv
import json
from common import time_matrix, cost_matrix
from statistics_3tasks import get_stats_for_result, sample_most_likely

In [2]:
M = [2,6]
K = [1,4]
T = [12, 6, 24]
d = 19
time_matrix = np.array(time_matrix(M, T))
cost_matrix = np.array(cost_matrix(time_matrix, K))
n1  = np.shape(cost_matrix)[0]
n2 = np.shape(cost_matrix)[1]

def createOp():
    mdl = Model(name='workflow')
    x = {(i, j) : mdl.binary_var(name='x_{0}_{1}'.format(i, j)) for i in range(0, n1) for j in range(0, n2)}
    x.update({i : mdl.binary_var(name='x_{0}'.format(i)) for i in range(7, 11)})

    workflow_func = mdl.sum(cost_matrix[i,j] * x[(i,j)] for i in range(n1) for j in range(n2))

    # machine usage constraint
    for j in range(n2):
        mdl.add_constraint(mdl.sum(x[(i,j)] for i in range(n1)) == 1)

    # time limit constraint
    mdl.add_constraint(mdl.sum(time_matrix[i, j] * x[(i, j)] for i in range(n1) for j in range(n2))+ 8 * x[7] + 4 * x[8] + 2 * x[9] + x[10] == d)
    mdl.minimize(workflow_func)
    qubitOp, offset = docplex.get_operator(mdl)
    return qubitOp

In [3]:
write_to_file = False
filename = 'results.txt'

seed = 1235691
aqua_globals.random_seed = seed
entanglement='full'

A = 1
B = 40
C = 1
reps = 2
max_trials = 1000
shots = 100

qubitOp = createOp()
spsa = SPSA(max_trials=max_trials)
ry = RealAmplitudes(qubitOp.num_qubits, reps=reps, entanglement=entanglement)
vqe = VQE(qubitOp, ry, spsa)
# provider= IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend,seed_simulator=seed, seed_transpiler=seed, shots=shots)
result = vqe.run(quantum_instance)   
state_vector = result['eigenstate']
get_stats_for_result(state_vector)

if write_to_file:
    with open(filename, 'a+') as file:
        file.write(json.dumps(state_vector)) 


most likely solution:  (array([1, 1, 0, 1, 1, 1, 0, 0, 1, 1]), 37)
optimal:  0
correct solutions:  0
incorrect solutions:  100
correct configs:  0


In [4]:
vqe.get_optimal_circuit().draw()

┌────────────────┐                                                       »
q_0: ┤ RY(-0.0039205) ├──■────■─────────■─────────■──────────────■────────────»
     └┬─────────────┬─┘┌─┴─┐  │         │         │              │            »
q_1: ─┤ RY(-1.4813) ├──┤ X ├──┼────■────┼────■────┼─────────■────┼─────────■──»
      └┬────────────┤  └───┘┌─┴─┐┌─┴─┐  │    │    │         │    │         │  »
q_2: ──┤ RY(-3.136) ├───────┤ X ├┤ X ├──┼────┼────┼────■────┼────┼────■────┼──»
       ├────────────┤       └───┘└───┘┌─┴─┐┌─┴─┐  │  ┌─┴─┐  │    │    │    │  »
q_3: ──┤ RY(1.2487) ├─────────────────┤ X ├┤ X ├──┼──┤ X ├──┼────┼────┼────┼──»
       ├───────────┬┘                 └───┘└───┘┌─┴─┐└───┘┌─┴─┐  │  ┌─┴─┐  │  »
q_4: ──┤ RY(-1.57) ├────────────────────────────┤ X ├─────┤ X ├──┼──┤ X ├──┼──»
       ├───────────┴┐                           └───┘     └───┘┌─┴─┐└───┘┌─┴─┐»
q_5: ──┤ RY(1.5672) ├──────────────────────────────────────────┤ X ├─────┤ X ├»
      ┌┴────────────┤                                          └───┘     └───┘»
q_6: ─┤ RY(-3.0991) ├─────────────────────────────────────────────────────────»
      └┬────────────┤                                                         »
q_7: ──┤ RY(4.0724) ├─────────────────────────────────────────────────────────»
      ┌┴────────────┤                                                         »
q_8: ─┤ RY(-1.2644) ├─────────────────────────────────────────────────────────»
      └┬────────────┤                                                         »
q_9: ──┤ RY(1.7592) ├─────────────────────────────────────────────────────────»
       └────────────┘                                                         »
«                                                                           »
«q_0: ──■───────────────────■───────────────────■────────────────────────■──»
«       │                   │                   │                        │  »
«q_1: ──┼──────────────■────┼──────────────■────┼───────────────────■────┼──»
«       │              │    │              │    │                   │    │  »
«q_2: ──┼─────────■────┼────┼─────────■────┼────┼──────────────■────┼────┼──»
«       │         │    │    │         │    │    │              │    │    │  »
«q_3: ──┼────■────┼────┼────┼────■────┼────┼────┼─────────■────┼────┼────┼──»
«       │  ┌─┴─┐  │    │    │    │    │    │    │         │    │    │    │  »
«q_4: ──┼──┤ X ├──┼────┼────┼────┼────┼────┼────┼────■────┼────┼────┼────┼──»
«       │  └───┘┌─┴─┐  │    │  ┌─┴─┐  │    │    │  ┌─┴─┐  │    │    │    │  »
«q_5: ──┼───────┤ X ├──┼────┼──┤ X ├──┼────┼────┼──┤ X ├──┼────┼────┼────┼──»
«     ┌─┴─┐     └───┘┌─┴─┐  │  └───┘┌─┴─┐  │    │  └───┘┌─┴─┐  │    │    │  »
«q_6: ┤ X ├──────────┤ X ├──┼───────┤ X ├──┼────┼───────┤ X ├──┼────┼────┼──»
«     └───┘          └───┘┌─┴─┐     └───┘┌─┴─┐  │       └───┘┌─┴─┐  │    │  »
«q_7: ────────────────────┤ X ├──────────┤ X ├──┼────────────┤ X ├──┼────┼──»
«                         └───┘          └───┘┌─┴─┐          └───┘┌─┴─┐  │  »
«q_8: ────────────────────────────────────────┤ X ├───────────────┤ X ├──┼──»
«                                             └───┘               └───┘┌─┴─┐»
«q_9: ─────────────────────────────────────────────────────────────────┤ X ├»
«                                                                      └───┘»
«     ┌───────────────┐                                            »
«q_0: ┤ RY(0.0040295) ├───────────────────────────────■────────────»
«     └───────────────┘               ┌────────────┐┌─┴─┐          »
«q_1: ─────────────────────────────■──┤ RY(2.7247) ├┤ X ├──────────»
«                                  │  └────────────┘└───┘          »
«q_2: ────────────────────────■────┼────────────────────────────■──»
«                             │    │                            │  »
«q_3: ───────────────────■────┼────┼───────────────────────■────┼──»
«                        │    │    │                       │    │  »
«q_4: ────────■──────────┼────┼────┼──────────────────■────┼────┼─